# Introduction

This project focuses on parsing jobs fetched in Excel format from LinkedIn and consists of two main parts:

1. **Extracting Job Requirements:** This section aims to analyze and extract key job requirements directly from the job descriptions available in the dataset.

2. **Matching Jobs with Resumes:** This section will identify the job that best aligns with a given resume, based on the extracted requirements and relevant resume details.

The results of these analyses will provide valuable insights into job matching and help streamline the recruitment process by efficiently identifying roles that are most suitable for specific candidates.


In [36]:
# install all library needed
!pip install openpyxl pandas python-dotenv langchain langchain-openai

In [37]:
# import all libraries needed
import logging
from typing import Dict, List

import openpyxl
import pandas as pd
from dotenv import load_dotenv
from langchain_community.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

from configs.log_config import setup_logging
from prompt.resume_job_matcher import Job, create_job_matcher_chain, get_chain_input
from prompt.job_requirement import create_job_requirement_chain

# Load logs and environment variable from dotenv

In [38]:
load_dotenv()
setup_logging()
logger = logging.getLogger(__name__)

## Load the linkedin jobs from excel file

In [39]:
def load_excel_data(file_path, sheet_name=None):
    """Load Excel data and return it as a pandas DataFrame."""
    wb = openpyxl.load_workbook(file_path, data_only=True)
    ws = wb[sheet_name] if sheet_name else wb.active
    data = ws.values
    columns = next(data)[0:]  # Extract the first row as column names
    return pd.DataFrame(data, columns=columns)
job_df = load_excel_data("linkedin_jobs_13052024.xlsx")

## Extract job requirements from job description

In [40]:
# init llm to generate result
llm = ChatOpenAI()

# OpenAI chain to extract job requirement
job_req_chain = create_job_requirement_chain(llm)

In [41]:
# Extract job requirement from description, synchronously
def get_job_requirement():
    with get_openai_callback() as cb:
        for idx, jd in enumerate(df["Job_description"]):
            df.at[idx, "Job_requirement"] = job_req_chain.invoke(jd)
        logger.info(cb)

In [120]:
import asyncio
# Define the semaphore with the desired concurrency limit
semaphore = asyncio.Semaphore(3)  # Limit to 5 concurrent tasks

async def limited_task(task_func, *args):
    async with semaphore:
        result = await task_func(*args)
        await asyncio.sleep(random.randint(2, 4))
        return result
        
# the same thing but asynchronously
async def aget_job_requirement():
    tasks = [
        limited_task(job_req_chain.ainvoke, jd)
        for idx, jd in enumerate(job_df["Job_description"]) 
    ]
    with get_openai_callback() as cb:
        results = await asyncio.gather(*tasks)
        logger.info(cb)
    return results

job_df["Job_requirement"] = await aget_job_requirement()


2024-05-13 23:54:28 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:54:29 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:54:29 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


CancelledError: 

In [43]:
from datetime import datetime
# Write results to an Excel file
date_str = datetime.now().strftime("%m%d%Y%H%M%S")
job_df.to_excel(f"linkedin_job_requirement_{date_str}.xlsx", index=False)

# Get the most matched job for every company

In [44]:
# job_df = load_excel_data("linkedin_job_requirement.xlsx")

In [117]:
# iterate companies and find the most matched jobs
# @dataclass
# class Job:
#     idx: int
#     position: str
#     requirements: str
#     link: str
# {
#     company_name: [Job, Job]
# }
def group_job_descriptions_by_company(
    df,
    job_title_col="Job_title",
    company_col="Company",
    job_req_col="Job_requirement",
    link_col="Job_link",
) -> Dict[str, List[Job]]:
    """Group job descriptions by company and return a dictionary."""
    company_jobs = {}
    for idx, row in df.iterrows():
        if row[company_col] not in company_jobs:
            company_jobs[row[company_col]] = []
        company_jobs[row[company_col]].append(
            Job(idx, row[job_title_col], row[job_req_col], link=row[link_col])
        )
    return company_jobs


company_jobs_map = group_job_descriptions_by_company(job_df)

# load resume
resume = open("resume.txt", encoding="utf-8").read()

# init llm to generate result
llm = ChatOpenAI(base_url="http://localhost:15000/v1", model="gpt-4o", default_headers={"Host": "api.openai.com"}, request_timeout=120)
job_matcher_chain = create_job_matcher_chain(llm)

In [121]:
import asyncio
import random

# Define the semaphore with the desired concurrency limit
semaphore = asyncio.Semaphore(5)  # Limit to 5 concurrent tasks
llm = ChatOpenAI(base_url="http://localhost:15000/v1", model="gpt-4o", default_headers={"Host": "api.openai.com"}, request_timeout=120)
job_matcher_chain = create_job_matcher_chain(llm)

async def limited_task(task_func, *args):
    async with semaphore:
        result = await task_func(*args)
        return result

async def aget_best_matched_job_by_company(n=0):
    n = n or len(company_jobs_map)
    tasks = [
        limited_task(job_matcher_chain.ainvoke, get_chain_input(jobs, resume))
        for _, jobs in company_jobs_map.items()
    ]
    with get_openai_callback() as cb:
        descriptions = await asyncio.gather(*tasks[:n])
        logger.info(cb)
    companies = list(company_jobs_map.keys())[:n]
    return pd.DataFrame({"company": companies, "description": descriptions})

def batch_job_match_result():
    chain_input = [get_chain_input(jobs, resume) for _, jobs in company_jobs_map.items()]
    job_matcher_chain.batch(chain_input, config={"max_concurrency": 3})

In [122]:
with get_openai_callback() as cb:
    batch_job_match_result()
    logger.info(cb)

2024-05-13 23:54:55 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:54:55 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:54:58 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:02 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:03 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:06 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:08 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:10 - httpx - INFO - HTTP Request: POST http://localhost:15000/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-13 23:55:15 - httpx - INFO - HTTP Request: POST http://l

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-EB9URWPvUxR4ZYrtCB5egFbP on tokens per min (TPM): Limit 30000, Used 25705, Requested 9219. Please try again in 9.848s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [104]:
best_job_df = await aget_best_matched_job_by_company()
best_job_df

CancelledError: 

In [66]:
best_job_df

,company,description
0,Konrad Group,1. **Software Developer (Entry Level):**\n -...
1,"Triunity Software, Inc.",1. **Full-stack Developer:**\n - Proven expe...
2,Street Context,1. **Software Developer 1 - Full Stack:**\n ...
3,Jerry,1. **Associate Software Engineer (Toronto):**\...
4,Scotiabank,1. **Full Stack Developer [Scotiabank]:**\n ...
...,...,...
116,NEARSOURCE TECHNOLOGIES,1. **Software Test Engineer - [REMOTE]:**\n ...
117,Klick,"1. **Full Stack Developer, AI Integration:**\n..."
118,Arista Networks,1. **Software Test Engineer:**\n - Your expe...
119,Techedin,1. **Senior Java Developer:**\n - Your exper...


In [95]:
from datetime import datetime
# Write results to an Excel file
date_str = datetime.now().strftime("%m%d%Y%H%M%S")
best_job_df.to_excel(f"best_job_{date_str}.xlsx", index=False)

print(f"Processing completed and data written to best_job.xlsx")

Processing completed and data written to best_job.xlsx


In [96]:
with open("best_job.md", "w") as f:
    for idx, row in best_job_df.iterrows():
        f.write("# " + row["company"] + "\n")
        f.write(row["description"]+ "\n")